# Composite Laminate Analysis Theory

This notebook explains the theory and mathematics behind the composite laminate analysis code in `custom_composite.py`. We'll go through each function and explain the underlying principles.

## 1. Material Properties and Coordinate Systems

### 1.1 Material Properties

For an orthotropic material (like carbon fiber), we need 4 independent elastic constants:
- $E_{11}$: Longitudinal modulus (along fibers)
- $E_{22}$: Transverse modulus (perpendicular to fibers)
- $\nu_{12}$: Major Poisson's ratio
- $G_{12}$: In-plane shear modulus

The minor Poisson's ratio $\nu_{21}$ is related to these by:
$$\nu_{21} = \nu_{12} \frac{E_{22}}{E_{11}}$$

This is implemented in the `PlyProperties` class:

In [ ]:
@dataclass
class PlyProperties:
    E11: float  # Longitudinal modulus (GPa)
    E22: float  # Transverse modulus (GPa)
    nu12: float  # Major Poisson's ratio
    G12: float  # In-plane shear modulus (GPa)
    thickness: float  # Ply thickness (mm)
    orientation: float  # Fiber orientation angle (degrees)
    density: float  # Density (kg/m³)
    
    @property
    def nu21(self) -> float:
        return self.nu12 * self.E22 / self.E11

### 1.2 Coordinate Systems

We work with two coordinate systems:
1. Material coordinates (1-2):
   - 1-axis: Along fibers
   - 2-axis: Perpendicular to fibers
2. Global coordinates (x-y):
   - x-axis: Reference direction
   - y-axis: Perpendicular to x

The transformation between these systems is handled by the transformation matrix $T$.

## 2. Stiffness Matrices

### 2.1 Reduced Stiffness Matrix (Q)

The reduced stiffness matrix $Q$ relates stresses to strains in material coordinates:
$$\begin{bmatrix} \sigma_1 \\ \sigma_2 \\ \tau_{12} \end{bmatrix} = 
\begin{bmatrix} Q_{11} & Q_{12} & 0 \\ Q_{12} & Q_{22} & 0 \\ 0 & 0 & Q_{66} \end{bmatrix}
\begin{bmatrix} \epsilon_1 \\ \epsilon_2 \\ \gamma_{12} \end{bmatrix}$$

Where:
$$Q_{11} = \frac{E_{11}}{1-\nu_{12}\nu_{21}}$$
$$Q_{22} = \frac{E_{22}}{1-\nu_{12}\nu_{21}}$$
$$Q_{12} = \frac{\nu_{12}E_{22}}{1-\nu_{12}\nu_{21}}$$
$$Q_{66} = G_{12}$$

Implemented in `_get_Q_matrix`:

In [ ]:
def _get_Q_matrix(self, ply: PlyProperties) -> np.ndarray:
    Q11 = ply.E11 / (1 - ply.nu12 * ply.nu21)
    Q22 = ply.E22 / (1 - ply.nu12 * ply.nu21)
    Q12 = ply.nu12 * ply.E22 / (1 - ply.nu12 * ply.nu21)
    Q66 = ply.G12
    
    return np.array([
        [Q11, Q12, 0],
        [Q12, Q22, 0],
        [0, 0, Q66]
    ])

### 2.2 Transformation Matrix (T)

The transformation matrix $T$ relates strains in global coordinates to material coordinates:
$$\begin{bmatrix} \epsilon_1 \\ \epsilon_2 \\ \gamma_{12} \end{bmatrix} = T \begin{bmatrix} \epsilon_x \\ \epsilon_y \\ \gamma_{xy} \end{bmatrix}$$

Where $T$ is:
$$T = \begin{bmatrix} 
\cos^2\theta & \sin^2\theta & 2\cos\theta\sin\theta \\
\sin^2\theta & \cos^2\theta & -2\cos\theta\sin\theta \\
-\cos\theta\sin\theta & \cos\theta\sin\theta & \cos^2\theta-\sin^2\theta
\end{bmatrix}$$

Implemented in `_get_T_matrix`:

In [ ]:
def _get_T_matrix(self, theta: float) -> np.ndarray:
    theta_rad = np.radians(theta)
    c = np.cos(theta_rad)
    s = np.sin(theta_rad)
    
    return np.array([
        [c**2, s**2, 2*c*s],
        [s**2, c**2, -2*c*s],
        [-c*s, c*s, c**2 - s**2]
    ])

### 2.3 Transformed Stiffness Matrix (Q̄)

The transformed stiffness matrix $\bar{Q}$ relates stresses to strains in global coordinates:
$$\begin{bmatrix} \sigma_x \\ \sigma_y \\ \tau_{xy} \end{bmatrix} = \bar{Q} \begin{bmatrix} \epsilon_x \\ \epsilon_y \\ \gamma_{xy} \end{bmatrix}$$

Where $\bar{Q} = T^{-1}QT$

Implemented in `_get_Q_bar_matrix`:

In [ ]:
def _get_Q_bar_matrix(self, ply: PlyProperties) -> np.ndarray:
    Q = self._get_Q_matrix(ply)
    T = self._get_T_matrix(ply.orientation)
    T_inv = np.linalg.inv(T)
    
    return T_inv @ Q @ T

## 3. Laminate Analysis

### 3.1 ABD Matrices

The ABD matrix relates forces and moments to strains and curvatures:
$$\begin{bmatrix} N \\ M \end{bmatrix} = \begin{bmatrix} A & B \\ B & D \end{bmatrix} \begin{bmatrix} \epsilon^0 \\ \kappa \end{bmatrix}$$

Where:
- $N$: Force resultants (N/m)
- $M$: Moment resultants (N·m/m)
- $\epsilon^0$: Mid-plane strains
- $\kappa$: Curvatures

The matrices are calculated as:
$$A = \sum_{k=1}^n \bar{Q}_k (z_k - z_{k-1})$$
$$B = \frac{1}{2}\sum_{k=1}^n \bar{Q}_k (z_k^2 - z_{k-1}^2)$$
$$D = \frac{1}{3}\sum_{k=1}^n \bar{Q}_k (z_k^3 - z_{k-1}^3)$$

Implemented in `calculate_ABD_matrices`:

In [ ]:
def calculate_ABD_matrices(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    A = np.zeros((3, 3))
    B = np.zeros((3, 3))
    D = np.zeros((3, 3))
    
    z_bottom = -self.total_thickness / 2
    
    for ply in self.plies:
        Q_bar = self._get_Q_bar_matrix(ply)
        z_top = z_bottom + ply.thickness
        
        A += Q_bar * ply.thickness
        B += Q_bar * (z_top**2 - z_bottom**2) / 2
        D += Q_bar * (z_top**3 - z_bottom**3) / 3
        
        z_bottom = z_top
    
    return A, B, D

### 3.2 Effective Properties

For a symmetric laminate (B = 0), the effective properties are calculated from the compliance matrix $S = A^{-1}$:

$$E_x = \frac{1}{hS_{11}}, \quad E_y = \frac{1}{hS_{22}}$$
$$\nu_{xy} = -\frac{S_{12}}{S_{11}}, \quad G_{xy} = \frac{1}{hS_{66}}$$

Where $h$ is the total laminate thickness.

For unsymmetric laminates, we use the full ABD matrix:
$$S = \begin{bmatrix} A & B \\ B & D \end{bmatrix}^{-1}$$

Implemented in `calculate_effective_properties`:

In [ ]:
def calculate_effective_properties(self) -> dict:
    A, B, D = self.calculate_ABD_matrices()
    h = self.total_thickness
    
    # Create the full ABD matrix
    ABD = np.block([[A, B], [B, D]])
    
    # Calculate compliance matrix
    S = np.linalg.inv(ABD)
    
    # Extract the in-plane compliance terms
    S11 = S[0,0]
    S12 = S[0,1]
    S22 = S[1,1]
    S66 = S[2,2]
    
    # Calculate effective properties
    E_x = 1 / (h * S11)
    E_y = 1 / (h * S22)
    nu_xy = -S12 / S11
    G_xy = 1 / (h * S66)
    
    return {
        'E_x': E_x / 1e9,  # Convert back to GPa
        'E_y': E_y / 1e9,
        'nu_xy': nu_xy,
        'G_xy': G_xy / 1e9,
        'thickness': h * 1000,  # Convert to mm
        'density': sum(ply.density * ply.thickness for ply in self.plies) / h
    }

### 3.3 Stress Analysis

To calculate stresses in each ply:

1. Solve for mid-plane strains and curvatures:
$$\begin{bmatrix} \epsilon^0 \\ \kappa \end{bmatrix} = \begin{bmatrix} A & B \\ B & D \end{bmatrix}^{-1} \begin{bmatrix} N \\ M \end{bmatrix}$$

2. Calculate strain at any point z:
$$\epsilon(z) = \epsilon^0 + z\kappa$$

3. Transform to material coordinates:
$$\epsilon_{12} = T\epsilon_{xy}$$

4. Calculate stresses in material coordinates:
$$\sigma_{12} = Q\epsilon_{12}$$

5. Transform back to global coordinates:
$$\sigma_{xy} = T^{-1}\sigma_{12}$$

Implemented in `get_ply_stresses`:

In [ ]:
def get_ply_stresses(self, loads: Dict[str, float]) -> List[Dict[str, np.ndarray]]:
    A, B, D = self.calculate_ABD_matrices()
    ABD = np.block([[A, B], [B, D]])
    
    # Create load vector
    N = np.array([loads.get('Nx', 0), loads.get('Ny', 0), loads.get('Nxy', 0)])
    M = np.array([loads.get('Mx', 0), loads.get('My', 0), loads.get('Mxy', 0)])
    load_vector = np.concatenate([N, M])
    
    # Calculate mid-plane strains and curvatures
    strains_curvatures = np.linalg.solve(ABD, load_vector)
    epsilon0 = strains_curvatures[:3]
    kappa = strains_curvatures[3:]
    
    # Calculate stresses in each ply
    z_bottom = -self.total_thickness / 2
    ply_stresses = []
    
    for ply in self.plies:
        z_top = z_bottom + ply.thickness
        z_mid = (z_top + z_bottom) / 2
        
        # Calculate strain at mid-plane of ply
        epsilon = epsilon0 + z_mid * kappa
        
        # Transform strain to material coordinates
        T = self._get_T_matrix(ply.orientation)
        epsilon_material = T @ epsilon
        
        # Calculate stress in material coordinates
        Q = self._get_Q_matrix(ply)
        sigma_material = Q @ epsilon_material
        
        # Transform stress back to global coordinates
        T_inv = np.linalg.inv(T)
        sigma_global = T_inv @ sigma_material
        
        ply_stresses.append({
            'ply_name': ply.name,
            'z_location': z_mid * 1000,  # Convert to mm
            'strain_material': epsilon_material,
            'strain_global': epsilon,
            'stress_material': sigma_material / 1e6,  # Convert to MPa
            'stress_global': sigma_global / 1e6
        })
        
        z_bottom = z_top
    
    return ply_stresses

## 4. Practical Example

Let's analyze a symmetric [0/45/-45/0]s layup using carbon fiber:

```python
sequence = [
    {'material_type': MaterialType.CARBON_T300, 'thickness': 0.1, 'orientation': 0},
    {'material_type': MaterialType.CARBON_T300, 'thickness': 0.1, 'orientation': 45},
    {'material_type': MaterialType.CARBON_T300, 'thickness': 0.1, 'orientation': -45},
    {'material_type': MaterialType.CARBON_T300, 'thickness': 0.1, 'orientation': 0}
]
```

This layup is symmetric (mirrored about the mid-plane), so:
- B matrix will be zero
- No coupling between extension and bending
- Effective properties will be the same in tension and compression

The analysis will show:
1. Higher stiffness in the 0° direction (E_x > E_y)
2. Reduced Poisson's ratio compared to unidirectional
3. Increased shear stiffness due to ±45° plies
4. Through-thickness stress distribution under applied loads

## 5. References

1. Jones, R. M. (1999). Mechanics of Composite Materials (2nd ed.). Taylor & Francis.
https://soaneemrana.com/onewebmedia/Mechanics%20of%20Composite%20Materials%202nd%20Ed%201999%20BY%20[Taylor%20&%20Francis].pdf

2. Tsai, S. W., & Hahn, H. T. (1980). Introduction to Composite Materials. Technomic Publishing.
https://www.researchgate.net/publication/287727399_Introduction_to_Composite_Materials_Technomic_Publishing_Lancaster


3. Daniel, I. M., & Ishai, O. (2006). Engineering Mechanics of Composite Materials (2nd ed.). Oxford University Press.
https://www.academia.edu/39988481/ENGINEERING_MECHANICS_OF_COMPOSITE_MATERIALS_SECOND_EDITION_Ori_lshai